
# 🛒 CMPE 256 — Market Basket Recommender (Colab v4, Wide → Tall)

```
transaction_id | item_1 | item_2 | item_3 | item_4 | item_5
```
Each row is one transaction and every `item_n` is a product bought together.

This notebook:
- 📥 Auto-prompts for upload if CSV missing
- 🔄 Converts **wide → tall** (melt) automatically
- 🧮 Builds Apriori rules (auto-tuned support)
- 🧯 Adds co-occurrence fallback recommender
- 🖥️ Gradio UI for live cart + recommendations


Submitted By Team :- **Bro Code**

In [1]:

# ⬇️ Install dependencies
!pip -q install mlxtend gradio pandas==2.2.2


In [2]:

# Imports & config
import os, re, sys, math
import pandas as pd
import numpy as np
from mlxtend.frequent_patterns import apriori, association_rules
import gradio as gr

pd.set_option("display.max_colwidth", 120)

DEFAULT_NAME = "CMPE256_Hackathon_market_basket_analysis_Release.csv"
ITEM_COLS = ['item_1','item_2','item_3','item_4','item_5']  # expected wide-format columns
MIN_LIFT = 0.8
TOP_K = 5


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)



## 📥 Upload CSV (auto-prompt if missing)


In [3]:

dataset_path = DEFAULT_NAME
if not os.path.exists(dataset_path):
    try:
        from google.colab import files
        print("File not found:", dataset_path)
        print("Please upload your CSV…")
        uploaded = files.upload()
        if uploaded:
            dataset_path = list(uploaded.keys())[0]
            print("Using uploaded file:", dataset_path)
        else:
            print("No file uploaded; will run with a tiny demo dataset.")
    except Exception as e:
        print("Upload not available (not running in Colab?):", e)
else:
    print("Found dataset:", dataset_path)


File not found: CMPE256_Hackathon_market_basket_analysis_Release.csv
Please upload your CSV…


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag

Saving CMPE256_Hackathon_market_basket_analysis_Release.csv to CMPE256_Hackathon_market_basket_analysis_Release.csv
Using uploaded file: CMPE256_Hackathon_market_basket_analysis_Release.csv


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag


## 🧼 Load & preview
Tries multiple encodings. If no CSV, builds a small demo that matches your schema.


In [4]:

def load_dataframe(path):
    if os.path.exists(path):
        last_err = None
        for enc in ("utf-8", "utf-8-sig", "latin-1"):
            try:
                df = pd.read_csv(path, encoding=enc)
                print(f"Loaded with encoding: {enc}")
                print("Columns:", list(df.columns))
                display(df.head(5))
                return df
            except Exception as e:
                last_err = e
        raise last_err
    else:
        print("⚠️ CSV not found. Using a tiny demo dataset in wide format.")
        demo = pd.DataFrame({
            "transaction_id": ["T1","T2","T3","T4","T5"],
            "item_1": ["Bosch D7050 Detector (SKU: D7050)", "DSC PG9914 Motion Detector (SKU: PG9914)",
                       "Axis M3046-V Network Camera (SKU: 0806-001)", "GE Interlogix 60-652-95R Carbon Monoxide Detector (SKU: 60-652-95R)",
                       "DSC WS4916 Smoke Detector (SKU: WS4916)"],
            "item_2": ["DSC PG9914 Motion Detector (SKU: PG9914)", "Bosch F220-B6 Detector Base (SKU: F220-B6)",
                       "Bosch NVR-5500-16A00 NVR (SKU: NVR-5500-16A00)", "Dahua 8-Channel NVR (SKU: NVR4208-8P-4KS2)",
                       "DSC WS4939 Wireless Key (SKU: WS4939)"],
            "item_3": ["", "", "", "", ""],
            "item_4": ["", "", "", "", ""],
            "item_5": ["", "", "", "", ""],
        })
        display(demo)
        return demo

df_raw = load_dataframe(dataset_path)


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag

Loaded with encoding: utf-8
Columns: ['transaction_id', 'item_1', 'item_2', 'item_3', 'item_4', 'item_5']


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


,transaction_id,item_1,item_2,item_3,item_4,item_5
0,1,Honeywell 5800CO Carbon Monoxide Detector (SKU: 5800CO),Hanwha XRN-2010 NVR (SKU: XRN-2010),NaN,NaN,NaN
1,2,Hanwha QNV-6010R Network Camera (SKU: QNV-6010R),Bosch B5512 Control Panel (SKU: B5512),Pelco IMP1110-1ES IP Camera (SKU: IMP1110-1ES),GE Interlogix 60-807-95R Glassbreak Detector (SKU: 60-807-95R),Bosch F220-B6 Detector Base (SKU: F220-B6)
2,3,Bosch F220-B6 Detector Base (SKU: F220-B6),GE Interlogix 60-652-95R Carbon Monoxide Detector (SKU: 60-652-95R),DSC WS4933 Carbon Monoxide Detector (SKU: WS4933),NaN,NaN
3,4,GE Interlogix 60-652-95R Carbon Monoxide Detector (SKU: 60-652-95R),Dahua 8-Channel NVR (SKU: NVR4208-8P-4KS2),Bosch NDN-50022-A3 IP Camera (SKU: NDN-50022-A3),Dahua DH-IPC-HDBW4431R-ZS IP Camera (SKU: DH-IPC-HDBW4431R-ZS),Bosch B5512 Control Panel (SKU: B5512)
4,5,Hanwha QNV-6010R Network Camera (SKU: QNV-6010R),Pelco DSSRV2-040-US NVR (SKU: DSSRV2-040-US),DSC WS4916 Smoke Detector (SKU: WS4916),Axis P3367-VE Network Camera (SKU: 0407-001),Dahua DH-IPC-HDBW4431R-ZS IP Camera (SKU: DH-IPC-HDBW4431R-ZS)


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag


## 🔄 Convert **wide → tall** (melt)
- Uses `transaction_id` as the transaction key
- Rows are melted from `item_1..item_5`
- Removes trailing `(SKU: …)` to reduce accidental uniqueness


In [5]:

# Validate required columns
required = ["transaction_id"] + ITEM_COLS
missing = [c for c in required if c not in df_raw.columns]
if missing:
    raise ValueError(f"Missing required columns for wide schema: {missing}\n"
                     f"Found: {list(df_raw.columns)}")

def normalize_item(x):
    x = str(x).strip()
    if not x or x.lower() == "nan":
        return ""
    x = re.sub(r"\s*\(SKU:\s*[^)]+\)\s*$", "", x)  # strip ' (SKU: ...)'
    x = re.sub(r"\s+", " ", x)
    return x.strip()

df_tall = df_raw.melt(
    id_vars=['transaction_id'],
    value_vars=ITEM_COLS,
    var_name='item_slot',
    value_name='Product_Name'
)
df_tall['Product_Name'] = df_tall['Product_Name'].astype(str).map(normalize_item)
df_tall = df_tall[df_tall['Product_Name'] != ""]

print("Tall shape:", df_tall.shape)
display(df_tall.head(10))


Tall shape: (3575, 3)


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag

,transaction_id,item_slot,Product_Name
0,1,item_1,Honeywell 5800CO Carbon Monoxide Detector
1,2,item_1,Hanwha QNV-6010R Network Camera
2,3,item_1,Bosch F220-B6 Detector Base
3,4,item_1,GE Interlogix 60-652-95R Carbon Monoxide Detector
4,5,item_1,Hanwha QNV-6010R Network Camera
5,6,item_1,Hanwha XRN-2010 NVR
6,7,item_1,Axis P3367-VE Network Camera
7,8,item_1,DSC PowerSeries Neo Alarm Kit
8,9,item_1,Honeywell 5800CO Carbon Monoxide Detector
9,10,item_1,Honeywell 5808W3 Smoke Detector


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag


## 🧺 Build one-hot basket


In [6]:

basket = (
    df_tall.assign(val=1)
    .pivot_table(index='transaction_id', columns='Product_Name', values='val', aggfunc='max', fill_value=0)
    .astype(int)
)
print("Basket shape:", basket.shape)
display(basket.head())

basket_sizes = basket.sum(axis=1)
print("Basket-size describe:\n", basket_sizes.describe())
print("Top items:\n", basket.sum(axis=0).sort_values(ascending=False).head(10))


Basket shape: (1000, 40)


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag

Product_Name,Axis M3046-V Network Camera,Axis P3225-LVE Network Camera,Axis P3367-VE Network Camera,Bosch B5512 Control Panel,Bosch B810 Wireless Receiver,Bosch D7050 Detector,Bosch DS160 Detector,Bosch F220-B6 Detector Base,Bosch NDN-50022-A3 IP Camera,Bosch NDN-832V03-IP IP Camera,...,Honeywell 5800CO Carbon Monoxide Detector,Honeywell 5800PIR-RES Motion Detector,Honeywell 5808W3 Smoke Detector,Honeywell 6160 Keypad,Honeywell H4D3PRV2 IP Camera,Honeywell HEN08104 NVR,Honeywell VISTA-20P Control Panel,Pelco DSSRV2-040-US NVR,Pelco IMP1110-1ES IP Camera,Pelco Sarix IMP121-1IS IP Camera
transaction_id,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
2,0,0,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
3,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


Basket-size describe:
 count    1000.00000
mean        3.57500
std         1.12856
min         2.00000
25%         3.00000
50%         4.00000
75%         5.00000
max         5.00000
dtype: float64
Top items:
 Product_Name
Dahua 8-Channel NVR                                  105
DSC PowerSeries Neo Alarm Kit                        104
Hikvision 4MP IP Camera                              101
DSC WS4939 Wireless Key                              101
Bosch D7050 Detector                                  99
GE Interlogix 60-848-02-95R Smoke Detector            99
Dahua DH-IPC-HDBW4431R-ZS IP Camera                   98
GE Interlogix 60-746-01-95R Wireless Keypad           98
Hikvision DS-7608NI-I2/8P NVR                         97
GE Interlogix 60-652-95R Carbon Monoxide Detector     97
dtype: int64


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag


## 📈 Apriori rules (auto-tuned support)
Tries several supports until rules appear.


In [7]:

def build_rules_with_autotune(basket, min_lift=0.8):
    N = basket.shape[0]
    supports = [0.05, 0.02, 0.01, 0.005, 0.002, 0.001]
    supports += [max(1, int(0.005*N))/N, max(1, int(0.001*N))/N, 1/max(N,1)]
    tried = []
    for s in supports:
        s = float(s)
        if s <= 0 or s > 1:
            continue
        fi = apriori(basket, min_support=s, use_colnames=True)
        if fi.empty:
            tried.append((s, 0, 0))
            continue
        rules = association_rules(fi, metric="lift", min_threshold=0.0)
        if rules.empty:
            tried.append((s, len(fi), 0))
            continue
        rules = rules[rules["lift"] >= min_lift].copy()
        if rules.empty:
            tried.append((s, len(fi), 0))
            continue
        to_list = lambda x: list(x) if not isinstance(x, list) else x
        rules["antecedents"] = rules["antecedents"].apply(to_list)
        rules["consequents"]  = rules["consequents"].apply(to_list)
        print(f"✅ Rules found with min_support={s:.6f} (itemsets={len(fi)}, rules={len(rules)})")
        return rules.sort_values(["lift","confidence"], ascending=False).reset_index(drop=True), s
    print("⚠️ No strong rules found. Tried supports:", tried)
    return pd.DataFrame(), None

rules, used_support = build_rules_with_autotune(basket, min_lift=MIN_LIFT)
print("Rules:", len(rules))
display(rules.head(10))


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag

✅ Rules found with min_support=0.010000 (itemsets=147, rules=214)
Rules: 214


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
0,[Honeywell 5800CO Carbon Monoxide Detector],[DSC WS4916 Smoke Detector],0.075,0.089,0.015,0.200000,2.247191,1.0,0.008325,1.138750,0.600000,0.100671,0.121844,0.184270
1,[DSC WS4916 Smoke Detector],[Honeywell 5800CO Carbon Monoxide Detector],0.089,0.075,0.015,0.168539,2.247191,1.0,0.008325,1.112500,0.609221,0.100671,0.101124,0.184270
2,[Bosch NDN-832V03-IP IP Camera],[Bosch NVR-5500-16A00 NVR],0.081,0.090,0.014,0.172840,1.920439,1.0,0.006710,1.100149,0.521530,0.089172,0.091032,0.164198
3,[Bosch NVR-5500-16A00 NVR],[Bosch NDN-832V03-IP IP Camera],0.090,0.081,0.014,0.155556,1.920439,1.0,0.006710,1.088289,0.526688,0.089172,0.081127,0.164198
4,[Pelco DSSRV2-040-US NVR],[GE Interlogix 60-807-95R Glassbreak Detector],0.080,0.082,0.012,0.150000,1.829268,1.0,0.005440,1.080000,0.492754,0.080000,0.074074,0.148171
5,[GE Interlogix 60-807-95R Glassbreak Detector],[Pelco DSSRV2-040-US NVR],0.082,0.080,0.012,0.146341,1.829268,1.0,0.005440,1.077714,0.493827,0.080000,0.072110,0.148171
6,[Axis M3046-V Network Camera],[Pelco IMP1110-1ES IP Camera],0.080,0.084,0.012,0.150000,1.785714,1.0,0.005280,1.077647,0.478261,0.078947,0.072052,0.146429
7,[Pelco IMP1110-1ES IP Camera],[Axis M3046-V Network Camera],0.084,0.080,0.012,0.142857,1.785714,1.0,0.005280,1.073333,0.480349,0.078947,0.068323,0.146429
8,[Dahua DH-IPC-HFW4431R-Z IP Camera],[Axis P3367-VE Network Camera],0.085,0.090,0.013,0.152941,1.699346,1.0,0.005350,1.074306,0.449769,0.080247,0.069166,0.148693
9,[Axis P3367-VE Network Camera],[Dahua DH-IPC-HFW4431R-Z IP Camera],0.090,0.085,0.013,0.144444,1.699346,1.0,0.005350,1.069481,0.452240,0.080247,0.064967,0.148693


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag


## 🔁 Co-occurrence fallback
Item–item counts ensure the UI still gives suggestions when rules are sparse.


In [8]:

co = basket.T.dot(basket)
for i in range(len(co)):
    co.iat[i, i] = 0
pop = basket.sum(axis=0)

def recommend_apriori(cart_items, top_k=TOP_K, rules_df=rules):
    if rules_df is None or rules_df.empty or not cart_items:
        return []
    cart_set = set(cart_items)
    cands = []
    for _, r in rules_df.iterrows():
        ant = set(r["antecedents"])
        if ant and ant.issubset(cart_set):
            for c in r["consequents"]:
                if c not in cart_set:
                    cands.append((c, float(r.get("confidence",0)), float(r.get("lift",0))))
    if not cands:
        return []
    ranked = sorted(cands, key=lambda x: (-x[2], -x[1], x[0]))
    out, seen = [], set()
    for item, conf, lift in ranked:
        if item not in seen:
            out.append(f"{item}  (lift={lift:.2f}, conf={conf:.2f})")
            seen.add(item)
        if len(out) >= top_k:
            break
    return out

def recommend_cooccurrence(cart_items, top_k=TOP_K):
    if not cart_items:
        return ["(Select items first)"]
    valid = [i for i in cart_items if i in co.index]
    if not valid:
        return ["(Selected items not in matrix — check data)"]
    scores = (co[valid].sum(axis=1)).sort_values(ascending=False)
    scores = scores[~scores.index.isin(valid)]
    if len(scores)==0 or scores.max()==0:
        return ["(no strong co-occurrence found — please inspect data)"]
    s = pd.concat([scores.rename("score"), pop.rename("pop")], axis=1).fillna(0)
    s = s.sort_values(by=["score","pop"], ascending=False).head(top_k)
    return [f"{idx}  (co_count={int(row['score'])}, pop={int(row['pop'])})" for idx,row in s.iterrows()]

def recommend_with_fallback(cart_items, top_k=TOP_K):
    via_rules = recommend_apriori(cart_items, top_k=top_k, rules_df=rules)
    if via_rules:
        return via_rules
    return recommend_cooccurrence(cart_items, top_k=top_k)


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag


## 🖥️ Gradio UI
Select items (multi-select) → **Get Recommendations**.


In [9]:

items = sorted(list(basket.columns))

with gr.Blocks(title="Market Basket Recommender") as demo:
    gr.Markdown("# 🛒 Market Basket Recommender (Wide → Tall)")
    gr.Markdown("Select items in your cart and click **Get Recommendations**.")
    cart = gr.CheckboxGroup(choices=items, label="Cart Items")
    btn = gr.Button("Get Recommendations")
    out = gr.Textbox(label="Recommended Items", lines=10)

    def _go(selected):
        return "\n".join(recommend_with_fallback(selected or []))

    btn.click(_go, inputs=cart, outputs=out)

demo.launch(share=True)


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag

* Running on public URL: https://55267bcc06659bb77b.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
